# Advanced Statistical Inference -- Variational Inference for Bayesian Logistic Regression

In this notebook, you will learn how to implement the (stochastic) variational inference algorithm.
The gist below will serve you as a refresh on variational inference. 
For additional information, please check the lecture notes. 

In the general setting, given a probabilistic model with observations $\{\boldsymbol{X},\boldsymbol{y}\}$, model parameters $\boldsymbol{w}$ and likelihood $p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w})$, by introducing an approximate posterior distribution $q_\theta(\boldsymbol{w})$ with parameters $\theta$, the variational lower bound to the log-marginal likelihood is defined as


$$\mathrm{KL}[{q_\theta(\boldsymbol{w})}||{p(\boldsymbol{w}|\boldsymbol{X},\boldsymbol{y})}] = -\mathbb{E}_{q_{\theta}}\log p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w}) +  \mathrm{KL}[{q_{\theta}(\boldsymbol{w})}||{p(\boldsymbol{w})}] + \log p(\boldsymbol{y}|\boldsymbol{X}) $$


The objective is then to maximize this variational bound (or evidence lower bound):

$$
      \mathcal{L}(\theta) = \underbrace{\mathbb{E}_{q_{\theta}}\log p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w})}_\text{Expected loglikelihood} -  \mathrm{KL}[{q_{\theta}(\boldsymbol{w})}||{p(\boldsymbol{w})}]
$$

The analytic evaluation of the ELBO is generally still untractable due to the presence of the expected loglikelihood under the variational distribution (in the majority of  cases the rightmost KL is tractable).
This is commonly overcome by sampling $N_\mathrm{MC}$ times from $q_\theta$  using the reparameterization trick

$$
    \mathbb{E}_{q_{\theta}}\log p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w}) \approx \dfrac{1}{N_\mathrm{MC}} \sum_{\tilde{\boldsymbol{w}}_i\sim q_\theta} \log p(\boldsymbol{y}|\boldsymbol{X}, \tilde{\boldsymbol{w}}_i)
$$

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rc
from tqdm.notebook import tqdm

colab = "google.colab" in str(get_ipython())
preamble = r"""\renewcommand{\familydefault}{\sfdefault}\usepackage{sansmath}
\usepackage{FiraSans}\sansmath\usepackage{amsmath}"""

rc("font", **{"family": "sans-serif", "sans-serif": "DejaVu Sans"})
rc("text", **{"usetex": not colab, "latex.preamble": preamble})
rc("figure", **{"dpi": 200})
rc(
    "axes",
    **{"spines.right": False, "spines.top": False, "xmargin": 0.0, "ymargin": 0.05}
)

## Library and coding

This lab is heavily built on JAX. You already used JAX during the lab on Gaussian process regression when you took advantage of the automatic differentation engine to optimize the kernel parameters, without manually writing the derivatives by hand. 
This time, we will this same auto-diff engine to optimize our variational objective, but we will also take advantage of another key charateristic of JAX: automatic vectorization. This will allow us to efficiently parallelize the computation of the expected likelihood. 
Below, you have a short tutorial on how this works.

Consider the following simple code that computes the convolution of two one-dimensional vectors:

In [ ]:
x = jnp.array([0.0, 1.0, 2.0, 3.0, 4.0])
w = jnp.array([2.0, 3.0, 4.0])


def convolve(x, w):
    y = []
    for i in range(1, len(x) - 1):
        y.append(jnp.dot(x[i - 1 : i + 2], w))
    return jnp.array(y)


y = convolve(x, w)
print(f"x = {x}\nw = {w}\ny = {y}")

Suppose we would like to apply this function to a batch of weights `w` to a batch of vectors `x`

In [ ]:
X = jnp.stack([x, 2 * x])
W = jnp.stack([w, 2 * w])

The most naive option would be to simply loop over the batch in Python and then stack the results; this produces the correct result, however it is not very efficient.
In order to batch the computation efficiently, you would normally have to rewrite the function manually to ensure it is done in vectorized form. 
This is not particularly difficult to implement for this particular example, but does involve changing how the function treats indices, axes, and other parts of the input.
Such re-implementation can become messy to understand and error-prone; fortunately JAX provides another way.
In JAX, the `jax.vmap` transformation is designed to generate such a vectorized implementation of a function automatically:

In [ ]:
Y = jax.vmap(convolve)(X, W)
print(f"X = \n{X}\nW =  \n{W}\nY =  \n{Y}")

Additional you can control what to vectorize using `in_axes=[...]`. For example, if you would like to convolve to a single set of weights `w` with a batch of vectors `X` you need to choose `in_axes=[0, None]`: this means "vectorize on the axis 0 of the first arg but do *not* vectorize on the second arg

In [ ]:
Y = jax.vmap(convolve, in_axes=[0, None])(X, w)
print(f"X = \n{X}\nw = \n{w}\nY =  \n{Y}")

Finally, this `vmap` can be also used as decorator, thus keeping the code cleaner

In [ ]:
@partial(jax.vmap, in_axes=[0, None])
def convolve(x, w):
    y = []
    for i in range(1, len(x) - 1):
        y.append(jnp.dot(x[i - 1 : i + 2], w))
    return jnp.array(y)


Y = convolve(X, w)
print(f"X = \n{X}\nw =\n {w}\nY =  \n{Y}")

### Random number generation in JAX 
To fully exploit the automatic vectorization of JAX, we will need to switch from the Numpy's random generation to JAX's. 
The main difference is that JAX is explicit on the definition and usage of the random seed. 

In [ ]:
rng = jax.random.PRNGKey(0)
print(jax.random.uniform(rng))

This also means that in order to have "true" random number generation, we need to manually advance the random seed. **The rule of thumb is: never reuse keys (unless you want identical outputs).**
In order to generate different and independent samples, you must `split()` the key *yourself* whenever you want to call a random function:

In [ ]:
print("rng", rng, "--> uniform", jax.random.uniform(rng))
rng, new_rng = jax.random.split(rng)
normal_sample = jax.random.uniform(new_rng)
print(r"    \---SPLIT --> new_rng", new_rng, "--> uniform", jax.random.uniform(new_rng))

While all this sounds exceedingly over-complicated, it also allows to very easily vectorize using `jax.vmap` functions that works on random samples. Take a look at the next cell and try to understand what is going on here. 

In [ ]:
def take_square(rng):
    x = jax.random.uniform(rng)
    return x**2


multiple_rng = jax.random.split(rng, 10)
print(jax.vmap(take_square)(multiple_rng))

# 1. Setup and data

Similarly to the previous lab, you’re going to implement the VI algorithm described in the lecture for binary classification.

In [ ]:
def plot_data(X, y, ax):
    mask = y == 1
    config = dict(edgecolor="black", linewidth=1, zorder=10)
    ax.scatter(*X[mask].T, label="Class 1", facecolor="tab:blue", **config)
    ax.scatter(*X[~mask].T, label="Class 0", facecolor="tab:orange", **config)


def get_grid(xlim=(-3, 3), ylim=None, N=100):
    if ylim is None:
        ylim = xlim
    x_grid = np.linspace(*xlim, N)
    y_grid = np.linspace(*ylim, N)
    xx, yy = np.meshgrid(x_grid, y_grid)
    X_plot = np.vstack((xx.flatten(), yy.flatten())).T
    return xx, yy, X_plot

In [ ]:
!wget https://raw.githubusercontent.com/srossi93/asi-labs/master/lab_week4-public/binaryclass2.csv

In [ ]:
data = np.loadtxt("binaryclass2.csv", delimiter=",")
X = data[..., :-1]
y = data[..., -1]

fig, ax = plt.subplots(figsize=[5, 3])
plot_data(X, y, ax)
ax.set_title("Binary classification")
ax.legend()
ax.set_xlim(-6, 6), ax.set_ylim(-6, 6)
plt.show()

Let's start by defining some probability distributions that we will need in this notebook. 
First, the Bernoulli distribution. 

**Exercise:**
Complete the following class to compute the logdensity of the Bernoulli distribution. Since we will be taking gradients of this likelihood, it's possible that during optimization $p$ becomes (down to machine precision) exactly 0 or 1, which makes the evaluation of the logarithm troublesome. To avoide this behavior, simply add a small quantity (like `1e-10`) before computing the log.

In [ ]:
def bernoulli_density(y, p):
    return # @@ COMPLETE @@ #

**Exercise:** 
Now let's move to the Gaussian distribution. Note that the parameter $\sigma^2$ is always expected to be positive while it is possible that the optimisation algorithm attempts to evaluate the log-likelihood in regions of the parameter space where one or more of these parameters are negative, leading to numerical issues. 
A commonly-used technique to enforce this condition is to work with a transformed version of parameters using the logarithm transformation. 
In particular, define $\psi = \log\sigma^2$. 
So remember to take the exponential of $\psi$ if you want to use $\sigma^2$.

In [ ]:
from functools import partial
from typing import NamedTuple


class GaussianDiagonal(NamedTuple):
    mean: jnp.array
    log_var: jnp.array

**Exercise**: Complete the next function to sample once using the reparameterization trick. Remember $p(\boldsymbol z) = p(t(\boldsymbol \varepsilon;\boldsymbol \theta))$ where $t(\cdot) = \boldsymbol \mu + \boldsymbol \sigma \odot \boldsymbol \varepsilon$ and $\boldsymbol \varepsilon \sim \mathcal{N}(\boldsymbol 0, \boldsymbol 1)$

In [ ]:
def sample_gaussian_diagonal(rng, params):
    mean, log_var = params
    eps = # @@ COMPLETE @@ #
    return # @@ COMPLETE @@ #

**Exercise**: Create a Gaussian with two components with $\mathcal{N}(\boldsymbol 0, \boldsymbol 1)$ and get one sample.

In [ ]:
rng = jax.random.PRNGKey(0)
p_params = # @@ COMPLETE @@ #
# @@ COMPLETE @@ #

# 2. KL Divergence

The expression of the KL divergence between multivariate Gaussians $q = \mathcal{N}(\boldsymbol{\mu}_q, \boldsymbol\Sigma_q)$ and $p = \mathcal{N}(\boldsymbol{\mu}_p, \boldsymbol\Sigma_p)$ is as follows:

\begin{equation}
\mathrm{KL}[q || p] = 
\frac{1}{2} \mathrm{Tr}(\boldsymbol\Sigma_p^{-1} \boldsymbol\Sigma_q)
+ \frac{1}{2} (\boldsymbol\mu_p - \boldsymbol\mu_q)^{\top} \boldsymbol\Sigma_1^{-1} (\boldsymbol\mu_p - \boldsymbol\mu_q)
- \frac{D}{2} 
+ \frac{1}{2} \log\left( \frac{\mathrm{det}\boldsymbol\Sigma_p}{\mathrm{det}\boldsymbol\Sigma_q} \right)
\end{equation}


This formula simplifies when the two Gaussians have diagonal covariance, i.e. $q = \mathcal{N}(\boldsymbol{\mu}_q, \boldsymbol{\sigma}^2_q\mathrm{I})$ and $p = \mathcal{N}(\boldsymbol{\mu}_p, \boldsymbol{\sigma}^2_p\mathrm{I})$,

$$
\mathrm{KL}[q || p] =  \frac{1}{2} \sum\left( \log \frac{\sigma^2_p}{\sigma^2_q} + \frac{\sigma_q^2 + (\mu_q - \mu_p)^2}{\sigma_p^2} - 1 \right)
$$


**Exercise:** 
Complete the next function to compute the KL divergence between two multivariate Gaussian distribution with diagonal covariance. *Note:* Since we have parameterized the Gaussian distribution with the logvariance, the formula above can be simplified even further.

In [ ]:
def kl_diag_diag(q_params, p_params):
    assert isinstance(q_params, GaussianDiagonal)
    assert isinstance(p_params, GaussianDiagonal)
    mean_q, log_var_q = q_params
    mean_p, log_var_p = p_params
    # @@ COMPLETE @@ #
    return kl

**Exercise:**
Create two identical Gaussian distributions and compute the KL divergence using the function. What's the result? Is it what you were expecting? 

In [ ]:
q_params = # @@ COMPLETE @@ #

print("KL =", kl_diag_diag(q_params, p_params))

# 3. Model

Now we can move to design the model. We will use a simple logistic regression very similarly to what done in the previous MCMC lab.
This very simple model computes $h(\boldsymbol{x}) = h(\boldsymbol{w}^\top\boldsymbol{x})$, where $h(\cdot)$ is the logistic function. 

**Exercise:** 
Complete the next two functions to compute (i) the logistic function and (ii) the output of the model, given a particular choice of $\boldsymbol w$.

In [ ]:
def logistic(z):
    return # @@ COMPLETE @@ #


def model(w, X):
    return # @@ COMPLETE @@ #

# 4. Variational objective

The objective is to maximize this variational bound:
$$
      \mathcal{L}(\theta) = \underbrace{\mathbb{E}_{q_{\theta}}\log p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w})}_\text{Expected loglikelihood} -\mathrm{KL}[{q_{\theta}(\boldsymbol{w})}||{p(\boldsymbol{w})}]
$$

**Exercise:**
Complete the next cell to compute the ELBO.

In [ ]:
def create_elbo_fn(sample_fn, likelihood_fn, kl_divergence_fn):
    """Create a function to compute the ELBO, given the function to sample
    from the posterior, the likelihood function and the KL divergence
    """

    @partial(jax.vmap, in_axes=[0, None, None, None])
    def likelihood_sample_fn(rng, q_params, X, y):
        """Compute the likelihood with one Monte Carlo sample of the posterior
        The function is decorated to vectorize multiple MC samples automatically
        """
        # Get one sample of w using the sample_fn and the parameters of q
        w = # @@ COMPLETE @@ #
        # Predict the output using the sample before
        yp = # @@ COMPLETE @@ #
        # Compute the likelihood and return it
        return # @@ COMPLETE @@ #

    def elbo_fn(q_params, p_params, rng, X, y, Nmc=1):
        """Computes the ELBO with multiple samples"""
        # Split the random seed in Nmc times
        rng = jax.random.split(rng, Nmc)

        # Compute the values of the likelihood
        likelihood_vals = # @@ COMPLETE @@ #
        # Compute the expectation (i.e. take the mean)
        expected_likelihood = # @@ COMPLETE @@ #
        # Compute the KL divergence
        kl = # @@ COMPLETE @@ #
        # Compute the ELBO
        elbo = # @@ COMPLETE @@ #
        # Return the ELBO and its two term (used later for logging)
        return elbo, (expected_likelihood, kl)

    return elbo_fn

**Exercise:**
Using the function above, create the function to compute the ELBO.

In [ ]:
elbo_fn = create_elbo_fn(
    sample_fn=# @@ COMPLETE @@ #
    kl_divergence_fn=# @@ COMPLETE @@ #
    likelihood_fn=# @@ COMPLETE @@ #
)

**Exercise:**
Try to compute the variational objective (use 10 Monte Carlo samples).

In [ ]:
elbo, (likelihood, kl) = # @@ COMPLETE @@ #
print("ELBO =", elbo)
print("Likelihood =", likelihood)
print("KL =", kl)

## 4.1 Analysis of the MC estimate of the ELBO

Ok, now that everything is done and ready we can start to make some analysis.

First of all, as we said we don't have an analytical formula for the variational objective (our loss). We can only access (unbiased) samples, hence the next question.

**Exercise:**
Try to sample using 100 different random seeds the ELBO with [2, 10, 100, 1000] MC samples and plot their distribution with boxplots.

In [ ]:
elbo_samples = pd.DataFrame()
n_repetition = 100

for Nmc in [2, 10, 100, 1000]:
    elbo_samples[Nmc] = np.stack(
        [
            elbo_fn(q_params, p_params, jax.random.PRNGKey(i), X, y, Nmc=Nmc)[0]
            for i in range(n_repetition)
        ]
    )

In [ ]:
fig, ax = plt.subplots(figsize=[8, 4])
sns.boxplot(data=elbo_samples, whis=np.inf)
ax.set_title("Samples of the MC estimate of the ELBO")
ax.set_xlabel("Number of MC samples")
ax.set_ylabel("ELBO")
ax.margins(0, 0.05)

plt.show()

We said that, in case of large datasets, this can be computationally challenging, due to the evaluation of the likelihood $N_\mathrm{MC}$ times.
But we know that the ELBO can be approximated even further using mini-batching.
Taking a random subset of data $\mathcal{B}$, the approximation becomes

$$
    \mathbb{E}_{q_{\theta}}\log p(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{w}) \approx \dfrac{1}{N_\mathrm{MC}} \frac{N}{|\mathcal{B}|}\sum_{\tilde{\boldsymbol{w}}_i\sim q_\theta} \sum_{\boldsymbol{X}_j, \boldsymbol{y}_j\sim\mathcal{B}} \log p(\boldsymbol{y}_j|\boldsymbol{X}_j, \tilde{\boldsymbol{w}}_i)
$$

This introduces even more variance in the estimate of the ELBO but it allows to scale to (virtually) any sized dataset.
You are free to extend this to our ELBO estimator, but for the simple example we are using today, this is not required.

# 5. Optimization

Ok, now we can move to the optimization of the ELBO: 
$$
\boldsymbol \theta_{t+1} = \boldsymbol \theta_t + \text{lr}\cdot(\nabla_{\boldsymbol \theta} \mathcal{L})(\boldsymbol \theta_t)
$$
Below you have a simple function to implement this update

In [ ]:
def sg_update(params, gradients, learning_rate=1e-3):
    updated_params = jax.tree_map(lambda p, g: p + learning_rate * g, params, gradients)
    return updated_params

And here the function to compute the gradient (this function will also be compiled using `jax.jit` for better optimization and faster running times).

In [ ]:
grad_elbo_fn = jax.grad(elbo_fn, has_aux=True)
grad_elbo_fn = jax.jit(grad_elbo_fn, static_argnames=("Nmc"))

**Exercise:**
Write the training loop to optimize the ELBO (use a learning rate of $10^{-3}$). At every step, store the value of the ELBO, of the expected likelihood and the KL.

In [ ]:
rng = jax.random.PRNGKey(0)
q_params = GaussianDiagonal(jnp.zeros(2), jnp.zeros(2))

elbo_summary = []
lik_summary = []
kl_summary = []

for i in tqdm(range(10000), desc="Training ELBO"):
    rng, rng2 = jax.random.split(rng)
    q_params_grad, (likelihood, kl) = # @@ COMPLETE @@ #
    q_params = # @@ COMPLETE @@ #

    lik_summary.append(likelihood)
    kl_summary.append(kl)
    elbo_summary.append(likelihood - kl)

In [ ]:
print("Converged posterior")
print("Mean =", q_params.mean)
print("Var =", np.exp(q_params.log_var))

**Exercise:**
Assess convergence of the optimization by plotting the three metrics.

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=[8, 2.5])

ax0.plot(elbo_summary, label="ELBO")
ax1.plot(
    lik_summary,
    color="tab:orange",
    label=r"$E_{q(\mathbf{w})} \log p(\mathbf{y}|\mathbf{X},\mathbf{w})$",
)
ax2.plot(
    kl_summary,
    color="tab:green",
    label=r"$\mathrm{KL}[{q(\mathbf{w})}||{p(\mathbf{w})}]$",
)

ax0.semilogx()
ax1.semilogx()
ax2.semilogx()

ax0.axhline(-0.4, color="xkcd:red", label=r"$\log p(\mathbf{y}|\mathbf{X})$")
ax0.legend(bbox_to_anchor=(1.05, -0.05), loc="lower right")
ax1.legend(bbox_to_anchor=(1.05, -0.05), loc="lower right")
ax2.legend(bbox_to_anchor=(1.05, -0.05), loc="lower right")

ax0.set_xlabel("Iteration")
ax1.set_xlabel("Iteration")
ax2.set_xlabel("Iteration")

plt.show()

**Question:**
Analyze the behaviour of these three values and comment the plots. Focus you analysis on the breakdown of the ELBO in its parts.

Now the predictions! 

\begin{equation}
\mathbb{E}_{q(\boldsymbol{w})}h(\boldsymbol{w}^\top\boldsymbol{x}_\mathrm{new}) = \int h(\boldsymbol{w}^\top\boldsymbol{x}_\mathrm{new}) q(\boldsymbol{w}) \mathrm{d}\boldsymbol{w}
\end{equation}

**Exercise**:
With 1000 samples, compute the probability $P (y_\mathrm{new} = 1 | \boldsymbol{x}_\mathrm{new}, \boldsymbol{X}, \boldsymbol{y})$ when $\boldsymbol{x}_\mathrm{new} = [2,-4]^\top$. Compare the result with the number you got from the previous lab.


In [ ]:
def predict_y(sample_fn, q_params, Xt, rng, Nmc=10):
    """Compute the outputs of the model by sampling the posterior,
    then take the expectation
    """

    def predict_y_single(rng):
        w = # @@ COMPLETE @@ #
        yp = # @@ COMPLETE @@ #
        return yp

    rng = jax.random.split(rng, Nmc)
    Xt = jnp.atleast_1d(Xt)
    yp = jax.vmap(predict_y_single)(rng)
    # Take the mean
    yp = # @@ COMPLETE @@ #
    return yp


yp = predict_y(sample_gaussian_diagonal, q_params, [2, -4], rng, Nmc=1000)
print(yp)

**Execise:**
Let's now plot the distribution at convergence and the predictions on a grid of points.

In [ ]:
import scipy.stats


def plot_gaussian(params, **kwargs):
    ax = kwargs.pop("ax", plt.gca())
    xx, yy, w_plot = get_grid(xlim=(-1.5, 5.5), N=250)
    if isinstance(params, GaussianDiagonal):
        cov = np.exp(params.log_var) * np.eye(2)
    else:
        cov = params.L @ params.L.T
    zz = (
        scipy.stats.multivariate_normal(mean=params.mean, cov=cov)
        .pdf(w_plot)
        .reshape(*xx.shape)
    )

    levels = np.linspace(1e-5, np.max(zz), 10)
    ax.contourf(xx, yy, zz, cmap="cividis", alpha=0.8, levels=levels)
    ax.contour(xx, yy, zz, cmap="cividis", levels=levels)

    ax.set_xlabel(r"$\boldsymbol{w}_0$")
    ax.set_ylabel(r"$\boldsymbol{w}_1$")


def plot_posterior(ax):
    xx, yy, w_plot = get_grid(xlim=(-1.5, 5.5), N=250)
    zz = np.zeros(len(w_plot))
    for i, w in enumerate(w_plot):
        zz[i] = bernoulli_density(y, model(w, X)).sum() -0.5* w.T@w
    zz = np.exp(zz).reshape(*xx.shape)
    levels = np.linspace(1e-5, np.max(zz), 10)
    ax.contourf(xx, yy, zz, cmap="cividis", alpha=0.8, levels=levels)
    ax.contour(xx, yy, zz, cmap="cividis", levels=levels)
    ax.set_xlabel(r"$\boldsymbol{w}_0$")
    ax.set_ylabel(r"$\boldsymbol{w}_1$")

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=[12, 4])

plot_gaussian(p_params, ax=ax0)
plot_posterior(ax=ax1)
plot_gaussian(q_params, ax=ax2)

ax0.set_title("Prior")
ax1.set_title("True posterior")
ax2.set_title("Variational approx.")

plt.show()

In [ ]:
def plot_decision_boundary(xx, yy, P, ax):
    P = P.reshape(*xx.shape)
    levels = [0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 1]
    cs = ax.contour(xx, yy, P, levels, colors="k", linewidths=1.8, zorder=100)
    ax.clabel(cs, inline=1, fontsize=10)
    cs = ax.contourf(xx, yy, P, levels, cmap="Purples_r", alpha=0.5)


xx, yy, Xt = get_grid((-7, 7), N=50)
ps = # @@ COMPLETE @@ #

fig, ax = plt.subplots(figsize=[5, 4])
plot_decision_boundary(xx, yy, ps, ax=ax)
plot_data(X, y, ax=ax)

ax.set_xlabel(r"$\boldsymbol{x}_0$")
ax.set_ylabel(r"$\boldsymbol{x}_1$")
ax.set_title("Predictive density")
plt.show()

**Exercise:**
At convergence, sample 1000 times the ELBO like we did before, and plot it with a boxplot.

In [ ]:
converged_elbos = pd.DataFrame()
n_repetition = 1000
converged_elbos["Diagonal posterior"] = # @@ COMPLETE @@ #

In [ ]:
fig, ax = plt.subplots(figsize=[4, 3])
sns.boxplot(
    data=converged_elbos,
    whis=np.inf,
)
ax.axhline(-0.4, color="xkcd:red", label=r"$p(\boldsymbol{y}|\boldsymbol{X})$")
ax.set_title("ELBO at convergence", y=1.02)
ax.legend()
plt.show()

# 6. Alternatives to Mean Field Variational Inference

As we saw from one of the previous question, the approximation that we used is very rough: it recoves some properties of the true posterior but fails to capture the strong correlation that exists in the parameter space. 
What we need to do it to increase the complexity and the expressiveness of the variational posterior. 
The first step that we can do is to introduce a non-diagonal covariance $\boldsymbol{w} \sim \mathcal{N}(\mu, \Sigma)$, where the covariance $\Sigma=LL^\top$ ($L$ is a lower triangular matrix).

Sampling from such distribution is possible again using the reparameterization trick,

\begin{equation}
\tilde{\boldsymbol{w}} = \mu + \mathrm{Tril}(L)\boldsymbol{\varepsilon} \quad \boldsymbol{\varepsilon} \sim \mathcal{N}(0, \mathrm{I})
\end{equation}

where $\mathrm{Tril}(\cdot)$ returns the lower triangular part of the matrix (the other elements are set to 0).

**Exercise:** 
Complete the next functions to model a full covariance Gaussian distribution.

In [ ]:
class GaussianFullCov(NamedTuple):
    mean: jnp.array
    L: jnp.array


def sample_gaussian_fullcov(key, params):
    mean, L = params
    eps = # @@ COMPLETE @@ #
    return # @@ COMPLETE @@ #

In [ ]:
q_params = GaussianFullCov(jnp.zeros(2), jnp.eye(2))

print(sample_gaussian_fullcov(jax.random.PRNGKey(1), q_params))

Now we need to compute the KL divergence between $q$ Gaussian with full covariance and $p$ Gaussian with diagonal covariance. 
Remember that the expression of the KL divergence between multivariate Gaussians $q = \mathcal{N}(\boldsymbol{\mu}_q, \Sigma_q)$ and $p = \mathcal{N}(\boldsymbol{\mu}_p, \Sigma_p)$ is as follows:

\begin{equation}
\mathrm{KL}[q || p] = 
\frac{1}{2} \mathrm{Tr}(\Sigma_p^{-1} \Sigma_q)
+ \frac{1}{2} (\mu_p - \mu_q)^{\top} \Sigma_p^{-1} (\mu_p - \mu_q)
- \frac{D}{2} 
+ \frac{1}{2} \log\left( \frac{\mathrm{det}\Sigma_p}{\mathrm{det}\Sigma_q} \right)
\end{equation}

**Question:**
Given that $q = \mathcal{N}(\boldsymbol{\mu}_q, LL^\top)$ and $p = \mathcal{N}(\boldsymbol{\mu}_p, \sigma^2_p\mathrm{I})$, write the simplified KL divergence. 

*Hints:* 

- $\mathrm{Tr}(\boldsymbol L \boldsymbol L^\top) = \sum_i\mathrm{diag}(\boldsymbol L)_i^2$ 

- $\log\mathrm{det}(\boldsymbol L\boldsymbol L^\top) = \sum_i\log\mathrm{diag}(\boldsymbol L)_i^2$


In [ ]:
def kl_full_diag(q_params, p_params):
    assert isinstance(q_params, GaussianFullCov)
    assert isinstance(p_params, GaussianDiagonal)

    mean_q, L_q = q_params
    mean_p, log_var_p = p_params
    kl = # @@ COMPLETE @@ #
    return 0.5 * (kl)

In [ ]:
print(kl_full_diag(q_params, p_params))

**Exercise:**
Create a new ELBO which uses this new distribution.

In [ ]:
elbo_fn = create_elbo_fn(
    sample_fn=# @@ COMPLETE @@ #
    kl_divergence_fn=# @@ COMPLETE @@ #
    likelihood_fn=bernoulli_density,
)

**Exercise:**
Train this new model exactly the same as before

In [ ]:
# @@ COMPLETE @@ #

In [ ]:
# @@ COMPLETE @@ #

**Exercise:**
Plot the train curves

In [ ]:
# @@ COMPLETE @@ #

**Exercise:**
Plot the densities.

In [ ]:
# @@ COMPLETE @@ #

**Exercise:**
Plot the predictions.

In [ ]:
# @@ COMPLETE @@ #

**Exercise:**
Plot the ELBO at convergence.

In [ ]:
converged_elbos["Full cov. posterior"] = # @@ COMPLETE @@ #
# @@ COMPLETE @@ #

**Question:**
Do you observe something interesting? 
Remember what the ELBO represents. It is the lower bound of the marginal distribution $p(\boldsymbol{y}|\boldsymbol{X})$. Check the first lab on Bayesian linear regression if you don't remember what the marginal distribution measures. Based solely on this value, which model would you choose? Why?    